<a href="https://colab.research.google.com/github/RMoulla/legrand/blob/main/TP_Moteur_Recherche_IAG_Correction_Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **TP – Moteur de recherche e-commerce avec IAG**

### **Objectif**
Concevoir, avec l'aide de l'intelligence artificielle générative (IAG), un moteur de recherche e-commerce, du brief à la simulation du moteur, en passant par la génération des user stories et du prototype visuel.

### **Déroulé synthétique**
1. Reformuler le brief avec l'IAG.
2. Générer des user stories avec l'IAG.
3. Décrire une maquette avec Galileo AI.
4. Simuler un moteur de recherche en Python avec l'IAG.

### **Étape 1 — Reformuler le brief avec l'IAG**
Brief de départ :
« Les utilisateurs se plaignent de ne pas trouver facilement leurs produits. Concevoir un moteur de recherche intelligent capable de comprendre des requêtes naturelles.»

Instruction :
Utiliser un outil d'IAG pour reformuler ce brief en une problématique produit IA claire et proposer trois hypothèses de valeur mesurables.

### **Étape 2 — Générer des user stories avec l'IAG**

Objectif :
Obtenir un backlog initial décrivant les fonctionnalités du moteur de recherche intelligent.

Instruction :
Demander à l'IAG de proposer plusieurs user stories.

Demander également deux critères d'acceptation par user story.

### **Étape 3 — Décrire une maquette avec Stitch AI**

Objectif :
Transformer les user stories en description visuelle du moteur de recherche.

Instruction :
Rédiger un texte à copier dans `Stitch` pour générer une interface correspondant au moteur décrit dans les user stories. La maquette doit inclure :
- une barre de recherche avec auto-suggestion et correction automatique
- un panneau de filtres (prix, catégories, note)
- une zone d'affichage des résultats produits (image, titre, prix, note)
- une reformulation de la requête en cas d'erreur ou de faute

### **Étape 4 — Simuler un moteur de recherche en Python avec l'IAG**

Objectif :
Produire, à l'aide de l'IAG, un script Python simple simulant un moteur de recherche e-commerce.

Instruction :
Demander à l'IAG de générer un code qui :
- permet de chercher des produits par mots-clés à partir d'une requête, en utilisant comme base de données le fichier `products.csv`.
- affiche les résultats les plus pertinents triés par similarité textuelle
- utilise, pour l'interface, le code HTML généré par `Stitch`.

Prompt à copier coller :
```
Tu es un développeur expert. Ta mission est de créer un prototype de moteur de recherche en Python qui permet à l'utilisateur de rechercher un produit avec une requête en langage naturel. Tu dois respecter les contraintes suivantes :

* L'application doit tourner dans l'environnement Google Colab (avec Gradio).
* L'interface doit utiliser le code HTML fourni ici. Tu ne dois changer des éléments de l'interface que si nécessaire.
* La base de données est le fichier products.csv dont un les premières lignes sont fournies ici.
* L'application doit afficher les photos des articles sélectionnées ainsi que leurs prix et leurs notes.
```

In [ ]:
import pandas as pd
dataset=pd.read_csv('products.csv')
dataset.head()

,title,brand,category,sub_category,description,selling_price,average_rating,discount,images,url
0,Solid Men Black Track Pants,REEB,Clothing and Accessories,Bottomwear,NaN,"2,309",4.0,30% off,https://rukminim1.flixcart.com/image/128/128/k...,https://www.flipkart.com/reebok-solid-men-blac...
1,Men Brief (Pack of 3),NaN,Clothing and Accessories,Innerwear and Swimwear,Experience yourself the most comfortable produ...,332,3.5,14% off,https://rukminim1.flixcart.com/image/128/128/j...,https://www.flipkart.com/tt-men-brief/p/itm289...
2,Printed Men Hooded Neck Yellow T-Shirt,ARBO,Clothing and Accessories,Topwear,t_shirt,474,4.3,49% off,https://rukminim1.flixcart.com/image/128/128/k...,https://www.flipkart.com/arbour-printed-men-ho...
3,Printed Men Round Neck Dark Blue T-Shirt,Free Authori,Clothing and Accessories,Topwear,NaN,419,3.9,30% off,https://rukminim1.flixcart.com/image/128/128/k...,https://www.flipkart.com/free-authority-printe...
4,Tripin Brass Cufflink & Tie Pin Set (Silver),NaN,Clothing and Accessories,Clothing Accessories,TRIPIN UNIQUE SHAPE SILVER CUFFLINK FOR MEN WI...,500,NaN,49% off,https://rukminim1.flixcart.com/image/128/128/c...,https://www.flipkart.com/tripin-brass-cufflink...


In [ ]:
# Installation des dépendances
!pip install gradio pandas scikit-learn -q

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import re

# Fonction pour charger et préparer les données
def load_data(csv_path='products.csv'):
    """Charge le fichier CSV et prépare les données"""
    df = pd.read_csv(csv_path)

    # Nettoyer les prix (enlever les virgules)
    if 'selling_price' in df.columns:
        df['selling_price'] = df['selling_price'].astype(str).str.replace(',', '').astype(float)

    # Créer un texte combiné pour la recherche
    df['search_text'] = (
        df['title'].fillna('') + ' ' +
        df['brand'].fillna('') + ' ' +
        df['category'].fillna('') + ' ' +
        df['sub_category'].fillna('') + ' ' +
        df['description'].fillna('')
    ).str.lower()

    return df

# Classe pour le moteur de recherche
class ProductSearchEngine:
    def __init__(self, df):
        self.df = df
        self.vectorizer = TfidfVectorizer(
            max_features=5000,
            stop_words='english',
            ngram_range=(1, 2)
        )
        self.tfidf_matrix = self.vectorizer.fit_transform(df['search_text'])

    def search(self, query, top_k=10):
        """Recherche les produits correspondant à la requête"""
        if not query or query.strip() == '':
            return self.df.head(top_k)

        query_vec = self.vectorizer.transform([query.lower()])
        similarities = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        top_indices = similarities.argsort()[-top_k:][::-1]

        results = self.df.iloc[top_indices].copy()
        results['similarity_score'] = similarities[top_indices]

        return results

# Fonction pour générer le HTML des résultats
def generate_results_html(results_df):
    """Génère le HTML pour afficher les résultats"""
    if results_df.empty:
        return "<div class='text-center text-black/50 dark:text-white/50 py-8'>Aucun résultat trouvé</div>"

    html_items = []
    for idx, row in results_df.iterrows():
        # Récupérer l'URL de l'image
        image_url = row.get('images', '')
        if pd.isna(image_url) or image_url == '' or '...' in str(image_url):
            image_url = 'https://via.placeholder.com/128x128/e5e7eb/6b7280?text=No+Image'
        else:
            image_url = str(image_url).strip()

        # Récupérer les informations
        title = str(row.get('title', 'Sans titre'))
        price = row.get('selling_price', 0)
        rating = row.get('average_rating', 'N/A')
        discount = row.get('discount', '')

        # Formater le prix
        price_str = f"${price:,.2f}" if not pd.isna(price) else "Prix non disponible"

        # Formater la note
        rating_str = f"⭐ {rating}" if not pd.isna(rating) and rating != 'N/A' else ""

        # Formater la réduction
        discount_str = str(discount) if not pd.isna(discount) and discount != '' else ""

        # Créer la carte de produit
        item_html = f'''
<div style="display: flex; align-items: center; gap: 1rem; background: white; border-radius: 0.5rem; padding: 1rem; box-shadow: 0 1px 3px rgba(0,0,0,0.1); margin-bottom: 1.5rem;">
    <div style="width: 112px; height: 112px; flex-shrink: 0; border-radius: 0.5rem; overflow: hidden; background: #f3f4f6; display: flex; align-items: center; justify-content: center;">
        <img src="{image_url}" alt="{title}" style="width: 100%; height: 100%; object-fit: cover;" onerror="this.src='https://via.placeholder.com/128x128/e5e7eb/6b7280?text=No+Image';" loading="lazy">
    </div>
    <div style="flex: 1; display: flex; flex-direction: column; gap: 0.25rem;">
        <p style="font-weight: 700; color: #111827; line-height: 1.25; margin: 0;">{title}</p>
        <p style="font-size: 0.875rem; color: #6b7280; margin: 0;">{price_str}</p>
        {'<p style="font-size: 0.75rem; color: #9ca3af; margin: 0;">' + rating_str + '</p>' if rating_str else ''}
        {'<p style="font-size: 0.75rem; color: #10b981; margin: 0;">' + discount_str + '</p>' if discount_str else ''}
    </div>
</div>
'''
        html_items.append(item_html)

    return '\n'.join(html_items)

# Fonction pour créer l'interface complète
def create_search_interface(search_query):
    """Crée l'interface HTML complète avec les résultats de recherche"""

    # Effectuer la recherche
    results = search_engine.search(search_query, top_k=15)
    results_html = generate_results_html(results)

    # Template HTML complet
    html_template = f"""
    <!DOCTYPE html>
    <html lang="fr">
    <head>
        <meta charset="utf-8"/>
        <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
        <title>Recherche de Produits</title>
        <script src="https://cdn.tailwindcss.com?plugins=forms,container-queries"></script>
        <script>
            tailwind.config = {{
                darkMode: "class",
                theme: {{
                    extend: {{
                        colors: {{
                            "primary": "#0572cc",
                            "background-light": "#f5f7f8",
                            "background-dark": "#0f1a23",
                        }},
                        fontFamily: {{
                            "display": ["Inter"]
                        }},
                        borderRadius: {{
                            "DEFAULT": "0.25rem",
                            "lg": "0.5rem",
                            "xl": "0.75rem",
                            "full": "9999px"
                        }},
                    }},
                }},
            }}
        </script>
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;700;900&display=swap" rel="stylesheet"/>
        <style>
            body {{ min-height: 100vh; }}
            .scrollbar-hide::-webkit-scrollbar {{ display: none; }}
            .scrollbar-hide {{ -ms-overflow-style: none; scrollbar-width: none; }}
        </style>
    </head>
    <body class="font-display bg-background-light dark:bg-background-dark">
        <div class="relative flex h-auto min-h-screen w-full flex-col justify-between overflow-x-hidden">
            <div class="flex-grow">
                <header class="flex items-center justify-between bg-background-light dark:bg-background-dark p-4 pb-2">
                    <div class="w-12"></div>
                    <h1 class="text-lg font-bold text-black dark:text-white flex-1 text-center">Recherche</h1>
                    <div class="flex w-12 items-center justify-end">
                        <button class="flex h-12 w-12 cursor-pointer items-center justify-center rounded-full text-black dark:text-white">
                            <svg fill="currentColor" height="24" viewBox="0 0 256 256" width="24" xmlns="http://www.w3.org/2000/svg">
                                <path d="M40,88H73a32,32,0,0,0,62,0h81a8,8,0,0,0,0-16H135a32,32,0,0,0-62,0H40a8,8,0,0,0,0,16Zm64-24A16,16,0,1,1,88,80,16,16,0,0,1,104,64ZM216,168H199a32,32,0,0,0-62,0H40a8,8,0,0,0,0,16h97a32,32,0,0,0,62,0h17a8,8,0,0,0,0-16Zm-48,24a16,16,0,1,1,16-16A16,16,0,0,1,168,192Z"></path>
                            </svg>
                        </button>
                    </div>
                </header>

                <div class="px-4 py-3">
                    <div class="relative flex items-center w-full">
                        <div class="absolute inset-y-0 left-0 flex items-center pl-4 text-black/50 dark:text-white/50">
                            <svg fill="currentColor" height="20" viewBox="0 0 256 256" width="20" xmlns="http://www.w3.org/2000/svg">
                                <path d="M229.66,218.34l-50.07-50.06a88.11,88.11,0,1,0-11.31,11.31l50.06,50.07a8,8,0,0,0,11.32-11.32ZM40,112a72,72,0,1,1,72,72A72.08,72.08,0,0,1,40,112Z"></path>
                            </svg>
                        </div>
                        <input class="w-full h-12 pl-12 pr-12 rounded-lg bg-white dark:bg-background-dark border border-black/10 dark:border-white/10 text-black dark:text-white placeholder:text-black/50 dark:placeholder:text-white/50 focus:outline-none focus:ring-2 focus:ring-primary"
                               type="text"
                               value="{search_query if search_query else ''}"
                               readonly/>
                    </div>
                </div>

                <main class="px-4 pb-20">
                    <h2 class="text-lg font-bold text-black dark:text-white pb-4">Résultats ({len(results)})</h2>
                    {results_html}
                </main>
            </div>

            <footer class="fixed bottom-0 left-0 right-0 bg-background-light/80 dark:bg-background-dark/80 backdrop-blur-lg">
                <div class="flex justify-around border-t border-black/10 dark:border-white/10 py-2">
                    <a class="flex flex-col items-center gap-1 text-black/60 dark:text-white/60" href="#">
                        <svg fill="currentColor" height="24" viewBox="0 0 256 256" width="24" xmlns="http://www.w3.org/2000/svg">
                            <path d="M218.83,103.77l-80-75.48a16,16,0,0,0-21.66,0l-80,75.48A16,16,0,0,0,32,115.55V208a16,16,0,0,0,16,16H96a16,16,0,0,0,16-16V168a8,8,0,0,1,8-8h24a8,8,0,0,1,8,8v40a16,16,0,0,0,16,16h48a16,16,0,0,0,16-16V115.55A16,16,0,0,0,218.83,103.77ZM208,208H176V168a24,24,0,0,0-24-24H104a24,24,0,0,0-24,24v40H48V115.55l80-75.47,80,75.47Z"></path>
                        </svg>
                    </a>
                    <a class="flex flex-col items-center gap-1 text-primary dark:text-primary" href="#">
                        <svg fill="currentColor" height="24" viewBox="0 0 256 256" width="24" xmlns="http://www.w3.org/2000/svg">
                            <path d="M229.66,218.34l-50.07-50.06a88.11,88.11,0,1,0-11.31,11.31l50.06,50.07a8,8,0,0,0,11.32-11.32ZM40,112a72,72,0,1,1,72,72A72.08,72.08,0,0,1,40,112Z"></path>
                        </svg>
                    </a>
                    <a class="flex flex-col items-center gap-1 text-black/60 dark:text-white/60" href="#">
                        <svg fill="currentColor" height="24" viewBox="0 0 256 256" width="24" xmlns="http://www.w3.org/2000/svg">
                            <path d="M178,32c-20.65,0-38.73,8.88-50,23.89C116.73,40.88,98.65,32,78,32A62.07,62.07,0,0,0,16,94c0,70,103.79,126.66,108.21,129a8,8,0,0,0,7.58,0C136.21,220.66,240,164,240,94A62.07,62.07,0,0,0,178,32ZM128,206.8C109.74,196.16,32,147.69,32,94A46.06,46.06,0,0,1,78,48c19.45,0,35.78,10.36,42.6,27a8,8,0,0,0,14.8,0c6.82-16.67,23.15-27,42.6-27a46.06,46.06,0,0,1,46,46C224,147.61,146.24,196.15,128,206.8Z"></path>
                        </svg>
                    </a>
                    <a class="flex flex-col items-center gap-1 text-black/60 dark:text-white/60" href="#">
                        <svg fill="currentColor" height="24" viewBox="0 0 256 256" width="24" xmlns="http://www.w3.org/2000/svg">
                            <path d="M230.92,212c-15.23-26.33-38.7-45.21-66.09-54.16a72,72,0,1,0-73.66,0C63.78,166.78,40.31,185.66,25.08,212a8,8,0,1,0,13.85,8c18.84-32.56,52.14-52,89.07-52s70.23,19.44,89.07,52a8,8,0,1,0,13.85-8ZM72,96a56,56,0,1,1,56,56A56.06,56.06,0,0,1,72,96Z"></path>
                        </svg>
                    </a>
                </div>
                <div class="pb-3"></div>
            </footer>
        </div>
    </body>
    </html>
    """

    return html_template

# Charger les données et initialiser le moteur de recherche
print("Chargement des données...")
df = load_data('products.csv')
print(f"✓ {len(df)} produits chargés")

print("Initialisation du moteur de recherche...")
search_engine = ProductSearchEngine(df)
print("✓ Moteur de recherche prêt")

# Créer l'interface Gradio
demo = gr.Interface(
    fn=create_search_interface,
    inputs=gr.Textbox(
        label="Rechercher un produit",
        placeholder="Ex: wireless headphones, blue t-shirt, running shoes...",
        value="wireless headphones"
    ),
    outputs=gr.HTML(label="Résultats"),
    title="🔍 Moteur de Recherche Produits",
    description="Recherchez des produits en langage naturel. Le système utilise TF-IDF pour trouver les meilleurs résultats.",
    examples=[
        ["wireless headphones"],
        ["black track pants"],
        ["blue t-shirt"],
        ["men's clothing"],
        ["sports accessories"]
    ],
    theme=gr.themes.Soft()
)

# Lancer l'application
if __name__ == "__main__":
    demo.launch(debug=True, share=True)

Chargement des données...
✓ 2000 produits chargés
Initialisation du moteur de recherche...
✓ Moteur de recherche prêt
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a966fbe854bca5682a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
